## Hands-on 4
practice data transformation, grouping, and sorting.

<img src="https://secure.static.meredith.com/crt/store/covers/magazines/nmo/9826_l.jpg">

In [ ]:
csvurl="https://gist.githubusercontent.com/clairehq/" + \
        "79acab35be50eaf1c383948ed3fd1129/raw/407a02139ae1e134992b90b4b2b8c329b3d73a6a/winemag-data-130k-v2.csv"
import pandas as pd
wine = pd.read_csv(csvurl)
wine.drop(wine.columns[0], axis="columns", inplace=True)
wine.head()
wine.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65499 entries, 0 to 65498
Data columns (total 13 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   country                65467 non-null  object 
 1   description            65499 non-null  object 
 2   designation            46588 non-null  object 
 3   points                 65499 non-null  int64  
 4   price                  60829 non-null  float64
 5   province               65467 non-null  object 
 6   region_1               54744 non-null  object 
 7   region_2               25170 non-null  object 
 8   taster_name            51856 non-null  object 
 9   taster_twitter_handle  49467 non-null  object 
 10  title                  65499 non-null  object 
 11  variety                65499 non-null  object 
 12  winery                 65499 non-null  object 
dtypes: float64(1), int64(1), object(11)
memory usage: 6.5+ MB


#### Question 1: ####  
What is the mean of the points column?

In [ ]:
mean_points = wine.points.mean()
mean_points

88.43403716087269

mean of points column is 88.43403716087269


#### Question 2: ####  
How many countries are present in this dataset? (Only count each country once)

In [ ]:
unique_countries = wine.country.nunique()
unique_countries

41

41 countries in this dataset


#### Question 3: ####
How many times does each country appeared in this dataset? Show each country and the corresponding count (show counts in ascending order)

In [ ]:
country_count = wine['country'].value_counts()
country_count.sort_values(ascending=True)

Slovakia                      1
Bosnia and Herzegovina        1
Armenia                       1
Luxembourg                    4
Switzerland                   4
India                         4
Ukraine                       5
Czech Republic                6
Macedonia                     6
Cyprus                        6
Serbia                        7
Peru                          8
Morocco                      11
Lebanon                      20
Moldova                      30
Brazil                       31
Mexico                       31
England                      36
Georgia                      37
Slovenia                     39
Turkey                       43
Croatia                      44
Uruguay                      61
Hungary                      61
Romania                      67
Bulgaria                     68
Canada                      108
Greece                      242
Israel                      259
New Zealand                 733
South Africa                737
Germany 

#### Question 4: ####
Create a variable `adjusted_price` containing the adjusted price which is the price subtracted by the average price. *This is called **"centering" transformation** - a method commonly used in the preprocessing step before applying various machine learning algorithms.*

In [ ]:
wine['adj_price'] = wine.price - wine.price.mean()
wine.adj_price
wine[['price', 'adj_price']]

,price,adj_price
0,NaN,NaN
1,15.0,-20.232932
2,14.0,-21.232932
3,13.0,-22.232932
4,65.0,29.767068
...,...,...
65494,45.0,9.767068
65495,22.0,-13.232932
65496,20.0,-15.232932
65497,31.0,-4.232932


#### Question 5: ####
What is the title of the wine that has the highest points-to-price ratio in the dataset?

In [ ]:
wine['points_to_price'] = wine.points / wine.price

highest_ppr = wine.groupby('title').points_to_price.max().idxmax()
print('The title of the wine that has the highest points-to-price ratio: '
 + highest_ppr)



The title of the wine that has the highest points-to-price ratio: Bandit NV Merlot (California)


 Bandit NV Merlot (California)

#### Question 6: ####
Create a series `flavor_counts` that contains two values: the number of wines that has the word "tart" in the `description` column and the number of wines that has the word "berries" in the `description` column. The index of the Series should be "Tart" and "Berries" for the corresponding values.

In [ ]:
# Look for the word "tart", not case sensitive, not a substring of another word.
# Same applies to the word "berries"
tart = wine['description'].str.contains(r'\bTart\b', case=False)
berries = wine['description'].str.contains(r'\bBerries\b', case=False)
tart_count = tart.sum()
berries_count = berries.sum()
#series flavour count with index as tart and berries
flavour_count = pd.Series([tart_count, berries_count], index=['Tart', 'Berries'])
flavour_count


Tart       3086
Berries    1192
dtype: int64

#### Question 7: ####
Let's convert the points into simple star ratings. A score of 90 or higher counts as 3 stars, a score of at least 80 but less than 90 is 2 stars. Any other score is 1 star.

Also, any wines from France should automatically get 3 stars, regardless of points.

Add this new column `star_ratings` to the dataframe with the number of stars for each wine in the dataset.

In [ ]:
def star_ratings(row):
    if row.points >= 90 or row.country == "France":
        return 3
    elif row.points >= 80:
        return 2
    else:
        return 1
wine['star_ratings'] = wine.apply(star_ratings, axis="columns")
wine[['title', 'country', 'points', 'star_ratings']]


,title,country,points,star_ratings
0,Nicosia 2013 Vulkà Bianco (Etna),Italy,87,2
1,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portugal,87,2
2,Rainstorm 2013 Pinot Gris (Willamette Valley),US,87,2
3,St. Julian 2013 Reserve Late Harvest Riesling ...,US,87,2
4,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,US,87,2
...,...,...,...,...
65494,William Fèvre 2005 Fourchaume Premier Cru (Ch...,France,90,3
65495,Tapestry 2005 Cabernet Sauvignon (McLaren Vale),Australia,90,3
65496,Fritz 2006 Estate Sauvignon Blanc (Dry Creek V...,US,90,3
65497,Hendry 2004 Block 24 Primitivo (Napa Valley),US,90,3


#### Question 8: ####
Who are the most common wine reviewers in the dataset? Create a Series whose index is the taster_twitter_handle category from the dataset, and whose values count how many reviews each person wrote.

In [ ]:
reviewer_counts = wine.groupby("taster_twitter_handle").taster_twitter_handle.count()
reviewer_counts.sort_values(ascending=False)
print('The most common wine reviewe is ' + reviewer_counts.idxmax())




The most common wine reviewe is @vossroger


#### Question 9: ####
What combination of countries and varieties are most common? Create a Series whose index is a MultiIndexof {country, variety} pairs. For example, a pinot noir produced in the US should map to {"US", "Pinot Noir"}. Sort the values in the Series in descending order based on wine count.

In [ ]:
country_variety_counts = wine.groupby(["country", "variety"]).size().sort_values(ascending=False)
country_variety_counts.head(1)
# the most common combination of countries and varieties is US and Pinot Noir
print('The most common combination is ' , country_variety_counts.idxmax())
country_variety_counts.sort_values(ascending=False)



The most common combination is  ('US', 'Pinot Noir')


country  variety                 
US       Pinot Noir                  4918
         Cabernet Sauvignon          3649
         Chardonnay                  3412
France   Bordeaux-style Red Blend    2380
Italy    Red Blend                   1870
                                     ... 
France   Petite Verdot                  1
         Nielluciu                      1
         Cabernet                       1
England  White Blend                    1
Uruguay  Tempranillo-Tannat             1
Length: 1304, dtype: int64

#### Question 10 #####
Create a Series whose index is reviewers and whose values is the average score given out by that reviewer. Hint: you will need the `taster_name` and `points` columns.

In [ ]:
reviewer_mean_ratings = wine.groupby('taster_name').points.mean()
reviewer_mean_ratings

taster_name
Alexander Peartree    86.014286
Anna Lee C. Iijima    88.380506
Anne Krebiehl MW      90.587903
Carrie Dykes          86.644444
Christina Pickard     89.500000
Fiona Adams           87.090909
Jeff Jenssen          88.273504
Jim Gordon            88.604331
Joe Czerwinski        88.519770
Kerin O’Keefe         88.827969
Lauren Buzzeo         87.831557
Matt Kettmann         90.021087
Michael Schachner     86.904541
Mike DeSimone         89.030303
Paul Gregutt          89.095032
Roger Voss            88.678957
Sean P. Sullivan      88.666243
Susan Kostrzewa       86.408094
Virginie Boone        89.229557
Name: points, dtype: float64